Check Asset Compliance: SINCE policy
================================


In [1]:
# Define a compliance policy that alerts when an asset has expired.

# Main function parses in a url to the Archivist and client credentials , which is
# a user authorization. The main function would initialize an archivist connection
# using the url and the credentials, called "arch", then call arch.access_policies.list()
# with suitable properties and attributes.

In [2]:
from json import dumps as json_dumps
from os import getenv
from time import sleep
from uuid import uuid4
from warnings import filterwarnings

from archivist.archivist import Archivist
from archivist.compliance_policy_requests import (
    CompliancePolicySince,
)
from archivist.logger import set_logger

filterwarnings("ignore", message="Unverified HTTPS request")

In [3]:
# URL, CLIENT, SECRET are environment variables that represent connection parameters.
#
# URL = represents the url to the RKVST application
# CLIENT = represents the client ID from an Application Registration
# SECRET = represents the client secret from an Application Registration
RKVST_URL = getenv("RKVST_URL")
APPREG_CLIENT = getenv("RKVST_APPREG_CLIENT")
APPREG_SECRET = getenv("RKVST_APPREG_SECRET")

In [4]:
"""
Main function of Asset and Event creation.

* Connect to RKVST with client ID and client secret
* Creates an Asset and two Events
* Prints response of Asset and Event creation
"""

# Optional call to set the logger level.  The argument can be either
# "INFO" or "DEBUG".  For more sophisticated logging control see our
# documentation.
set_logger("INFO")

# Initialize connection to RKVST
print("Connecting to RKVST")
print("URL", RKVST_URL)
arch = Archivist(RKVST_URL, (APPREG_CLIENT, APPREG_SECRET), max_time=300)

Connecting to RKVST
URL https://app.dev-paul-0.wild.jitsuin.io


In [5]:
def create_compliance_policy(arch, tag):
    """Compliance policy which expires 10 seconds after a
    Maintenance Performed event on a 'Traffic Light' has occurred.

    Usually the expiry time is on the order of days or weeks..

    Additionally the use of tag is simply to make this example
    repeatable.
    """
    compliance_policy = arch.compliance_policies.create(
        CompliancePolicySince(
            description="Maintenance should be performed every 10 seconds",
            display_name="Regular Maintenance of Traffic light",
            asset_filter=[
                ["attributes.arc_display_type=Traffic Light"],
            ],
            event_display_type=f"Maintenance Performed {tag}",
            time_period_seconds=10,  # very short so we can test
        )
    )
    print("SINCE_POLICY:", json_dumps(compliance_policy, indent=4))
    return compliance_policy

In [6]:
def create_traffic_light(arch):
    """
    Creates a traffic light.

    Note that arc_display_type siginfies a Traffic Light
    """

    traffic_light = arch.assets.create(
        attrs={
            "arc_display_name": "Traffic light model 54",
            "arc_description": "Traffic flow control light at A603 North East",
            "arc_display_type": "Traffic Light",
        },
        confirm=True,
    )
    print("TRAFFIC_LIGHT:", json_dumps(traffic_light, indent=4))
    return traffic_light

In [7]:
def perform_maintenance(arch, traffic_light, tag):
    """
    Perform maintenance on traffic light
    """
    maintenance_performed = arch.events.create(
        traffic_light["identity"],
        {
            "operation": "Record",
            "behaviour": "RecordEvidence",
        },
        {
            "arc_description": "Maintenance performed on traffic light",
            "arc_display_type": f"Maintenance Performed {tag}",
        },
        confirm=True,
    )
    print("MAINTENANCE_PERFORMED:", json_dumps(maintenance_performed, indent=4))

In [8]:
tag = uuid4()  # make this example repeatable

In [9]:
# make a SINCE compliance policy that alerts when the
# maintenance performed event has expired.
compliance_policy = create_compliance_policy(arch, tag)
print("compliance_policy", json_dumps(compliance_policy, indent=4))

Refresh token


SINCE_POLICY: {
    "identity": "compliance_policies/e9b458b7-5e08-41ce-bf4c-ff01b7975148",
    "compliance_type": "COMPLIANCE_SINCE",
    "description": "Maintenance should be performed every 10 seconds",
    "display_name": "Regular Maintenance of Traffic light",
    "asset_filter": [
        {
            "or": [
                "attributes.arc_display_type=Traffic Light"
            ]
        }
    ],
    "event_display_type": "Maintenance Performed 281b5ac5-764f-446d-84da-17138e18fde0",
    "closing_event_display_type": "",
    "time_period_seconds": "10",
    "dynamic_window": "0",
    "dynamic_variability": 0,
    "richness_assertions": []
}
compliance_policy {
    "identity": "compliance_policies/e9b458b7-5e08-41ce-bf4c-ff01b7975148",
    "compliance_type": "COMPLIANCE_SINCE",
    "description": "Maintenance should be performed every 10 seconds",
    "display_name": "Regular Maintenance of Traffic light",
    "asset_filter": [
        {
            "or": [
                "attr

In [10]:
# create an asset that matches the assets_filter field in the
# compliance policy.
traffic_light = create_traffic_light(arch)
print("traffic_light", json_dumps(traffic_light, indent=4))

TRAFFIC_LIGHT: {
    "identity": "assets/b961c23f-c1db-4bde-9ed3-c997378f2bf2",
    "behaviours": [
        "Builtin",
        "AssetCreator",
        "Attachments",
        "RecordEvidence"
    ],
    "attributes": {
        "arc_description": "Traffic flow control light at A603 North East",
        "arc_display_name": "Traffic light model 54",
        "arc_display_type": "Traffic Light"
    },
    "confirmation_status": "CONFIRMED",
    "tracked": "TRACKED",
    "owner": "0x3E06AcBf002E8F67bDe832b3f3E7d2aDA2BE3DC8",
    "at_time": "2023-01-10T16:20:33Z",
    "storage_integrity": "TENANT_STORAGE",
    "proof_mechanism": "SIMPLE_HASH",
    "chain_id": "99",
    "public": false,
    "tenant_identity": "tenant/f9040e8b-2afa-48f1-bb00-f055d55ade01"
}
traffic_light {
    "identity": "assets/b961c23f-c1db-4bde-9ed3-c997378f2bf2",
    "behaviours": [
        "Builtin",
        "AssetCreator",
        "Attachments",
        "RecordEvidence"
    ],
    "attributes": {
        "arc_description"

In [11]:
# perform maintenance on the asset which is valid for 10 seconds.
perform_maintenance(arch, traffic_light, tag)

# and check compliance - should be OK.
print("Sleep 1 second...")
sleep(1)
compliance = arch.compliance.compliant_at(
    traffic_light["identity"],
)
print("COMPLIANCE (true):", json_dumps(compliance, indent=4))

MAINTENANCE_PERFORMED: {
    "identity": "assets/b961c23f-c1db-4bde-9ed3-c997378f2bf2/events/433f4c4e-f6e2-43f6-a48c-0f15152a2734",
    "asset_identity": "assets/b961c23f-c1db-4bde-9ed3-c997378f2bf2",
    "event_attributes": {
        "arc_description": "Maintenance performed on traffic light",
        "arc_display_type": "Maintenance Performed 281b5ac5-764f-446d-84da-17138e18fde0"
    },
    "asset_attributes": {},
    "operation": "Record",
    "behaviour": "RecordEvidence",
    "timestamp_declared": "2023-01-10T16:20:37Z",
    "timestamp_accepted": "2023-01-10T16:20:37Z",
    "timestamp_committed": "2023-01-10T16:20:37.759462037Z",
    "principal_declared": {
        "issuer": "https://app.dev-paul-0.wild.jitsuin.io/appidpv1",
        "subject": "bd3f85d2-9b92-4ef0-82c9-2bcb7b5d25c6",
        "display_name": "Application display name 05f19055-d2fd-4076-bd02-46a924cbe984",
        "email": ""
    },
    "principal_accepted": {
        "issuer": "https://app.dev-paul-0.wild.jitsuin.io

In [12]:
# however waiting long enough (> 10s) will cause the asset to
# become non-compliant...
print("Sleep 15 seconds...")
sleep(15)
compliance = arch.compliance.compliant_at(
    traffic_light["identity"],
)
print("COMPLIANCE (false):", json_dumps(compliance, indent=4))

Sleep 15 seconds...
COMPLIANCE (false): {
    "compliant": false,
    "compliance": [
        {
            "compliance_policy_identity": "compliance_policies/e9b458b7-5e08-41ce-bf4c-ff01b7975148",
            "compliant": false,
            "reason": "Duration 20s exceeds limit 10s"
        }
    ],
    "next_page_token": "",
    "compliant_at": "2023-01-10T16:20:58Z"
}


In [13]:
# finally delete the compliance_policy
_ = arch.compliance_policies.delete(
    compliance_policy["identity"],
)